In [1]:
import pandas as pd
import numpy as np
import plotly.express as px
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.graph_objects as go
from sklearn.cluster import KMeans
from sklearn.metrics import pairwise_distances_argmin_min
sns.set_theme(style = 'darkgrid')

In [2]:
Google_covid = 'https://drive.google.com/uc?id=1--nOkGMwXdgHEnw-RRLp056eYlpfce_h'
data = pd.read_csv(Google_covid)

In [3]:
data.date = pd.to_datetime(data['date'], format="%Y-%m-%d")
data

,date,country_code,country_name,iso_3166_1_alpha_3,new_confirmed,new_deceased,new_persons_vaccinated,new_persons_fully_vaccinated,new_vaccine_doses_administered,population,population_male,population_female,population_rural,population_urban,population_density,population_age_00_09,population_age_10_19,population_age_20_29,population_age_30_39,population_age_40_49,population_age_50_59,population_age_60_69,population_age_70_79,population_age_80_and_older,gdp_per_capita_usd,school_closing,workplace_closing,cancel_public_events,restrictions_on_gatherings,public_transport_closing,stay_at_home_requirements,restrictions_on_internal_movement,international_travel_controls,testing_policy,emergency_investment_in_healthcare,investment_in_vaccines,facial_coverings,life_expectancy,adult_male_mortality_rate,adult_female_mortality_rate,...,new_confirmed_age_4,new_confirmed_age_5,new_confirmed_age_6,new_confirmed_age_7,new_confirmed_age_8,new_confirmed_age_9,new_deceased_age_0,new_deceased_age_1,new_deceased_age_2,new_deceased_age_3,new_deceased_age_4,new_deceased_age_5,new_deceased_age_6,new_deceased_age_7,new_deceased_age_8,new_deceased_age_9,new_hospitalized_patients_age_0,new_hospitalized_patients_age_1,new_hospitalized_patients_age_2,new_hospitalized_patients_age_3,new_hospitalized_patients_age_4,new_hospitalized_patients_age_5,new_hospitalized_patients_age_6,new_hospitalized_patients_age_7,new_hospitalized_patients_age_8,new_hospitalized_patients_age_9,new_deceased_male,new_deceased_female,new_recovered_age_0,new_recovered_age_1,new_recovered_age_2,new_recovered_age_3,new_recovered_age_4,new_recovered_age_5,new_recovered_age_6,new_recovered_age_7,new_recovered_age_8,new_recovered_age_9,new_recovered_male,new_recovered_female
0,2020-01-01,AD,Andorra,AND,0.0,0.0,NaN,NaN,NaN,77265.0,58625.0,55581.0,9269.0,67873.0,164.394,9370.0,12022.0,10727.0,12394.0,21001.0,20720.0,14433.0,8657.0,4881.0,40886.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2020-01-02,AD,Andorra,AND,0.0,0.0,NaN,NaN,NaN,77265.0,58625.0,55581.0,9269.0,67873.0,164.394,9370.0,12022.0,10727.0,12394.0,21001.0,20720.0,14433.0,8657.0,4881.0,40886.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2020-01-03,AD,Andorra,AND,0.0,0.0,NaN,NaN,NaN,77265.0,58625.0,55581.0,9269.0,67873.0,164.394,9370.0,12022.0,10727.0,12394.0,21001.0,20720.0,14433.0,8657.0,4881.0,40886.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2020-01-04,AD,Andorra,AND,0.0,0.0,NaN,NaN,NaN,77265.0,58625.0,55581.0,9269.0,67873.0,164.394,9370.0,12022.0,10727.0,12394.0,21001.0,20720.0,14433.0,8657.0,4881.0,40886.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2020-01-05,AD,Andorra,AND,0.0,0.0,NaN,NaN,NaN,77265.0,58625.0,55581.0,9269.0,67873.0,164.394,9370.0,12022.0,10727.0,12394.0,21001.0,20720.0,14433.0,8657.0,4881.0,40886.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,..

In [4]:
group = data.groupby("country_name", dropna=True).sum()
group

,new_confirmed,new_deceased,new_persons_vaccinated,new_persons_fully_vaccinated,new_vaccine_doses_administered,population,population_male,population_female,population_rural,population_urban,population_density,population_age_00_09,population_age_10_19,population_age_20_29,population_age_30_39,population_age_40_49,population_age_50_59,population_age_60_69,population_age_70_79,population_age_80_and_older,gdp_per_capita_usd,school_closing,workplace_closing,cancel_public_events,restrictions_on_gatherings,public_transport_closing,stay_at_home_requirements,restrictions_on_internal_movement,international_travel_controls,testing_policy,emergency_investment_in_healthcare,investment_in_vaccines,facial_coverings,life_expectancy,adult_male_mortality_rate,adult_female_mortality_rate,infant_mortality_rate,mobility_retail_and_recreation,mobility_grocery_and_pharmacy,mobility_parks,...,new_confirmed_age_4,new_confirmed_age_5,new_confirmed_age_6,new_confirmed_age_7,new_confirmed_age_8,new_confirmed_age_9,new_deceased_age_0,new_deceased_age_1,new_deceased_age_2,new_deceased_age_3,new_deceased_age_4,new_deceased_age_5,new_deceased_age_6,new_deceased_age_7,new_deceased_age_8,new_deceased_age_9,new_hospitalized_patients_age_0,new_hospitalized_patients_age_1,new_hospitalized_patients_age_2,new_hospitalized_patients_age_3,new_hospitalized_patients_age_4,new_hospitalized_patients_age_5,new_hospitalized_patients_age_6,new_hospitalized_patients_age_7,new_hospitalized_patients_age_8,new_hospitalized_patients_age_9,new_deceased_male,new_deceased_female,new_recovered_age_0,new_recovered_age_1,new_recovered_age_2,new_recovered_age_3,new_recovered_age_4,new_recovered_age_5,new_recovered_age_6,new_recovered_age_7,new_recovered_age_8,new_recovered_age_9,new_recovered_male,new_recovered_female
country_name,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
Afghanistan,157387.0,7310.0,4397449.0,3510568.0,5228706.0,2.752234e+10,1.412322e+10,1.339912e+10,1.996885e+10,6.926672e+09,42156.289,7.839734e+09,6.943842e+09,4.974361e+09,3.205995e+09,2.095166e+09,1.301020e+09,7.476497e+08,339681685.0,74888975.0,354914.0,1018.0,971.0,774.0,700.0,417.0,352.0,370.0,1110.0,1201.0,248977750.0,1.130000e+08,1457.0,45591.602,167950.678,136120.124,33865.3,14838.0,29558.0,6153.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Albania,200639.0,3104.0,1080490.0,971973.0,2130714.0,2.023736e+09,1.010723e+09,1.013013e+09,7.823648e+08,1.235548e+09,70417.200,2.255853e+08,2.697502e+08,3.353725e+08,2.593325e+08,2.401658e+08,2.815776e+08,2.230656e+08,134862371.0,36922368.0,3783864.0,1235.0,949.0,1120.0,2208.0,262.0,1076.0,272.0,1105.0,1267.0,26000000.0,6.900000e+06,1722.0,55782.300,65973.705,34986.602,5514.6,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Algeria,210921.0,6083.0,6740064.0,4655573.0,12145830.0,3.100269e+10,1.566274e+10,1.533995e+10,8.160868e+09,2.227764e+10,13016.577,6.841985e+09,4.759345e+09,4.647617e+09,5.143591e+09,3.852174e+09,2.686640e+09,1.789093e+09,875148638.0,407094135.0,2791236.0,994.0,1037.0,1025.0,2322.0,915.0,1184.0,712.0,2026.0,889.0,326000000.0,1.642714e+09,1665.0,54221.951,72715.657,57035.104,14210.7,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
American Samoa,10.0,0.0,0.0,0.0,0.0,3.902357e+07,0.000000e+00,0.000000e+00,5.026063e+06,3.407952e+07,195121.395,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.0,0.0,8106462.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000e+00,0.0,0.000,0.000,0.000,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,

# Poblaciones por edades

In [5]:
group_pop = data.iloc[:,:24].groupby("country_name", dropna=True).first()
group_pop

,date,country_code,iso_3166_1_alpha_3,new_confirmed,new_deceased,new_persons_vaccinated,new_persons_fully_vaccinated,new_vaccine_doses_administered,population,population_male,population_female,population_rural,population_urban,population_density,population_age_00_09,population_age_10_19,population_age_20_29,population_age_30_39,population_age_40_49,population_age_50_59,population_age_60_69,population_age_70_79,population_age_80_and_older
country_name,,,,,,,,,,,,,,,,,,,,,,,
Afghanistan,2020-01-01,AF,AFG,0.0,0.0,8200.0,21936.0,8200.0,38928341.0,19976265.0,18952076.0,28244481.0,9797273.0,59.627,11088732.0,9821559.0,7035871.0,4534646.0,2963459.0,1840198.0,1057496.0,480455.0,105925.0
Albania,2020-01-01,AL,ALB,0.0,0.0,128.0,437.0,128.0,2862427.0,1429594.0,1432833.0,1106598.0,1747593.0,99.600,319074.0,381542.0,474360.0,366807.0,339697.0,398271.0,315510.0,190753.0,52224.0
Algeria,2020-01-01,DZ,DZA,0.0,0.0,30.0,2275188.0,30.0,43851043.0,22153807.0,21697235.0,11542954.0,31510100.0,18.411,9677490.0,6731747.0,6573715.0,7275235.0,5448619.0,3800056.0,2530542.0,1237834.0,575805.0
American Samoa,2020-01-01,AS,ASM,0.0,0.0,NaN,NaN,NaN,55196.0,NaN,NaN,7109.0,48203.0,275.985,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Andorra,2020-01-01,AD,AND,0.0,0.0,460.0,92.0,460.0,77265.0,58625.0,55581.0,9269.0,67873.0,164.394,9370.0,12022.0,10727.0,12394.0,21001.0,20720.0,14433.0,8657.0,4881.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Wallis and Futuna,2020-01-01,WF,WLF,0.0,0.0,1904.0,0.0,1904.0,11245.0,NaN,NaN,NaN,NaN,80.329,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Western Sahara,2020-01-01,EH,ESH,NaN,NaN,NaN,NaN,NaN,597330.0,312260.0,285070.0,NaN,NaN,2.246,112594.0,98369.0,104985.0,108715.0,82794.0,51951.0,28374.0,7612.0,1936.0
Yemen,2020-01-01,YE,YEM,0.0,0.0,85515.0,4745.0,85515.0,29825967.0,15024985.0,14800983.0,18292399.0,10869523.0,56.492,8033408.0,6750274.0,5732746.0,4140438.0,2362804.0,1422835.0,879450.0,403471.0,100542.0


In [6]:
AgesClassPop = ['0-9','10-19', '20-29','30-39','40-49','50-59','60-69','70-79','80+']
paises = ['Argentina', 'Brazil', 'Colombia', 'Czech Republic', 'Estonia',
       'Finland', 'France', 'Germany', 'Hong Kong', 'India', 'Malaysia',
       'Mexico', 'New Zealand', 'Peru', 'Philippines', 'Romania', 'Spain',
       'Taiwan', 'Thailand','India', 'Malaysia']

In [7]:
def plot_bar(pais, data, AgeClass, title, color):
  fig = go.Figure(go.Bar(
              x=data.loc[pais,:].values,
              y=AgeClass,
              orientation='h',
              marker=dict(color= color),))
  
  
  fig.update_layout(title= title + ' ' + pais, #f'Nuevos casos de Covid-19 por edad en el país {pais}'
                    xaxis_title='Población',
                    yaxis_title='Edades', 
                    autosize=False,
                    width=600,
                    height=600,
                    margin=dict(
                        l=50,
                        r=50,
                        b=100,
                        t=100,
                        pad=4
                    ))
  fig.show()

In [8]:
for pais in paises:
  plot_bar(pais, group_pop.iloc[:,14:], AgesClassPop, 'Población por edad de', '#123456')

# Casos confirmados covid

In [9]:
new_confirmed = group.iloc[:,43:53][group.new_confirmed_age_4 !=0]
AgeClass = ['0-9','10-19', '20-29','30-39','40-49','50-59','60-69','70-79','80-89','90+']
new_confirmed.loc[:,'ochenta_mas'] = new_confirmed.new_confirmed_age_8	 + new_confirmed.new_confirmed_age_9
new_confirmed

,new_confirmed_age_0,new_confirmed_age_1,new_confirmed_age_2,new_confirmed_age_3,new_confirmed_age_4,new_confirmed_age_5,new_confirmed_age_6,new_confirmed_age_7,new_confirmed_age_8,new_confirmed_age_9,ochenta_mas
country_name,,,,,,,,,,,
Argentina,131102.0,417201.0,1082889.0,1213884.0,1027630.0,706784.0,418378.0,215182.0,96337.0,30053.0,126390.0
Brazil,891485.0,1122459.0,3112221.0,3970246.0,3382726.0,2433207.0,1392404.0,641819.0,252615.0,63043.0,315658.0
Colombia,174762.0,389758.0,1078198.0,1137278.0,860235.0,698339.0,420897.0,206197.0,94605.0,20795.0,115400.0
Czech Republic,175287.0,275102.0,257035.0,337260.0,447025.0,323357.0,210848.0,142517.0,62287.0,15958.0,78245.0
Estonia,15215.0,30197.0,26636.0,40762.0,37029.0,29611.0,23845.0,11220.0,4410.0,0.0,4410.0
Finland,23240.0,32056.0,39094.0,35148.0,27681.0,18765.0,9815.0,4928.0,3738.0,0.0,3738.0
France,442199.0,1079958.0,1339956.0,1249608.0,1131151.0,972615.0,649900.0,391816.0,263711.0,138722.0,402433.0
Germany,173323.0,734232.0,1813864.0,2284900.0,804326.0,371117.0,0.0,0.0,0.0,0.0,0.0
Hong Kong,545.0,761.0,1864.0,2310.0,2075.0,1936.0,1759.0,785.0,345.0,87.0,432.0


In [10]:
def plot_bar_sub(pais, data_pob, data_new, AgeClass, title, color, labelx):
  fig = go.Figure(go.Bar(
              x=data_pob.loc[pais,:].values/data_pob.sum(axis=1)[pais],
              y=AgeClass,
              orientation='h',
              marker=dict(color= '#123456'),name='Población total'))

  fig.add_trace(go.Bar(
              x=data_new.loc[pais][[0,1,2,3,4,5,6,7,-1]].values/data_new.sum(axis=1)[pais],
              y=AgeClass,
              orientation='h',
              marker=dict(color= color), name= labelx))
  
  fig.update_layout(title= title + ' ' + pais, #barmode='relative' f'Nuevos casos de Covid-19 por edad en el país {pais}'
                    xaxis_title='Población',
                    yaxis_title='Edades', 
                    autosize=False,
                    width=600,
                    height=600,
                    margin=dict(
                        l=50,
                        r=50,
                        b=100,
                        t=100,
                        pad=4
                    ))
  fig.show()

In [11]:
def plot_bar(pais, data, AgeClass, title):
  fig = go.Figure(go.Bar(
              x=data.loc[pais,:].values,
              y=AgeClass,
              orientation='h'))
  
  fig.update_layout(title= title + ' ' + pais, #f'Nuevos casos de Covid-19 por edad en el país {pais}'
                    xaxis_title='Población',
                    yaxis_title='Edades', 
                    autosize=False,
                    width=600,
                    height=600,
                    margin=dict(
                        l=50,
                        r=50,
                        b=100,
                        t=100,
                        pad=4
                    ))
  fig.show()

In [13]:
for pais in group.new_confirmed_age_1[group.new_confirmed_age_1 !=0].index:
  plot_bar(pais, new_confirmed, AgeClass, 'Nuevos casos de Covid-19 por edad en el país')

In [15]:
for pais in group.new_confirmed_age_1[group.new_confirmed_age_1 !=0].index:
  plot_bar_sub(pais, group_pop.iloc[:,14:], new_confirmed, AgesClassPop, 'Nuevos casos de Covid-19 por edad en el país', 'red', 'Nuevos casos')

# Casos de muertes por covid 19

In [16]:
new_deceased = group.iloc[:,53:63][group.new_deceased_age_4 !=0]
new_deceased.loc[:,'ochenta_mas'] = new_deceased.new_deceased_age_8	 + new_deceased.new_deceased_age_9
new_deceased

,new_deceased_age_0,new_deceased_age_1,new_deceased_age_2,new_deceased_age_3,new_deceased_age_4,new_deceased_age_5,new_deceased_age_6,new_deceased_age_7,new_deceased_age_8,new_deceased_age_9,ochenta_mas
country_name,,,,,,,,,,,
Argentina,136.0,167.0,836.0,2565.0,6912.0,14681.0,25699.0,30790.0,24812.0,10021.0,34833.0
Brazil,1161.0,1201.0,7309.0,25144.0,52128.0,90462.0,124924.0,124726.0,86508.0,26849.0,113357.0
Colombia,336.0,426.0,2297.0,5444.0,11252.0,22161.0,34424.0,36453.0,28576.0,8704.0,37280.0
Czech Republic,4.0,4.0,27.0,124.0,443.0,1311.0,4911.0,12091.0,11225.0,3628.0,14853.0
Germany,13.0,16.0,265.0,4975.0,31009.0,66835.0,0.0,0.0,0.0,0.0,0.0
Hong Kong,0.0,0.0,0.0,2.0,4.0,9.0,29.0,61.0,80.0,28.0,108.0
India,1.0,0.0,1.0,2.0,5.0,5.0,17.0,9.0,3.0,0.0,3.0
Malaysia,64.0,80.0,688.0,2269.0,4254.0,5935.0,7240.0,5824.0,3507.0,753.0,4260.0
Mexico,598.0,617.0,4333.0,13994.0,33506.0,60186.0,79651.0,65218.0,31589.0,5511.0,37100.0


In [18]:
for country in new_deceased.index:
  plot_bar(country, new_deceased, AgeClass, 'Muertes a causa de Covid-19 por edad en el país')

In [19]:
for country in new_deceased.index:
  plot_bar_sub(country, group_pop.iloc[:,14:], new_deceased, AgesClassPop, 'Muertes a causa de Covid-19 por edad en el país', 'orange', 'Muertes')

# Casos de hospitalizaciones

In [20]:
new_hospitalized = group.iloc[:,63:73][group.new_hospitalized_patients_age_4 !=0]
new_hospitalized.loc[:,'ochenta_mas'] = new_hospitalized.new_hospitalized_patients_age_8	 + new_hospitalized.new_hospitalized_patients_age_9
new_hospitalized

,new_hospitalized_patients_age_0,new_hospitalized_patients_age_1,new_hospitalized_patients_age_2,new_hospitalized_patients_age_3,new_hospitalized_patients_age_4,new_hospitalized_patients_age_5,new_hospitalized_patients_age_6,new_hospitalized_patients_age_7,new_hospitalized_patients_age_8,new_hospitalized_patients_age_9,ochenta_mas
country_name,,,,,,,,,,,
Argentina,33958.0,15218.0,29295.0,40600.0,47151.0,55771.0,65471.0,66650.0,51590.0,20266.0,71856.0
Brazil,19889.0,14433.0,70424.0,191585.0,285052.0,353061.0,338209.0,267812.0,156345.0,42064.0,198409.0
Hong Kong,539.0,756.0,1840.0,2282.0,2055.0,1917.0,1719.0,720.0,264.0,59.0,323.0
India,78.0,194.0,463.0,508.0,362.0,283.0,208.0,59.0,13.0,4.0,17.0
Mexico,6354.0,5881.0,26357.0,56369.0,94177.0,129836.0,136302.0,98055.0,44398.0,7474.0,51872.0
Philippines,2339.0,3313.0,12394.0,13343.0,11442.0,13857.0,13347.0,8284.0,3197.0,480.0,3677.0
Spain,3421.0,4499.0,15203.0,27557.0,48183.0,67743.0,77333.0,82063.0,112454.0,0.0,112454.0


In [21]:
for pais in new_hospitalized.index:
  plot_bar(pais, new_hospitalized, AgeClass, 'Hospitalizaciones por Covid-19 por edad en el país')

In [22]:
for country in new_hospitalized.index:
  plot_bar_sub(country, group_pop.iloc[:,14:], new_hospitalized, AgesClassPop, 'Hospitalizaciones por Covid-19 por edad en el país', '#0FB2A4', 'Hospitalizaciónes')

#Casos de Recuperados

In [23]:
new_recovered = group.iloc[:,75:85][group.new_recovered_age_3 !=0]
AgeClass = ['0-9','10-29','30-39','40-49','50-59','60-69','70-79','80-89','90-99', '99+']
new_recovered

,new_recovered_age_0,new_recovered_age_1,new_recovered_age_2,new_recovered_age_3,new_recovered_age_4,new_recovered_age_5,new_recovered_age_6,new_recovered_age_7,new_recovered_age_8,new_recovered_age_9
country_name,,,,,,,,,,
Colombia,173658.0,388494.0,1073676.0,1129336.0,847101.0,674895.0,385886.0,169737.0,66330.0,12199.0
Czech Republic,137074.0,213810.0,229766.0,292786.0,391235.0,291930.0,186398.0,119506.0,47983.0,11731.0
Germany,146057.0,548738.0,1569850.0,1924122.0,657518.0,270786.0,0.0,0.0,0.0,0.0
Philippines,17570.0,31119.0,105384.0,94532.0,63751.0,49696.0,29583.0,12630.0,3981.0,509.0


In [24]:
for pais in new_recovered.index:
  plot_bar(pais, new_recovered, AgeClass, 'Recuperados de Covid-19 por edad en el país')

In [25]:
for country in new_recovered.index:
  plot_bar_sub(country, group_pop.iloc[:,14:], new_recovered, AgesClassPop, 'Recuperados de Covid-19 por edad en el país', '#5DB009', 'Recuperaciones')

# Rates

In [26]:
group_rate = data.loc[:,['date','country_name','population', 'population_male', 'population_female','adult_male_mortality_rate','adult_female_mortality_rate', 'infant_mortality_rate']].groupby("country_name", dropna=True).first()
group_rate

,date,population,population_male,population_female,adult_male_mortality_rate,adult_female_mortality_rate,infant_mortality_rate
country_name,,,,,,,
Afghanistan,2020-01-01,38928341.0,19976265.0,18952076.0,237.554,192.532,47.9
Albania,2020-01-01,2862427.0,1429594.0,1432833.0,93.315,49.486,7.8
Algeria,2020-01-01,43851043.0,22153807.0,21697235.0,102.851,80.672,20.1
American Samoa,2020-01-01,55196.0,NaN,NaN,NaN,NaN,NaN
Andorra,2020-01-01,77265.0,58625.0,55581.0,NaN,NaN,2.7
...,...,...,...,...,...,...,...
Wallis and Futuna,2020-01-01,11245.0,NaN,NaN,NaN,NaN,NaN
Western Sahara,2020-01-01,597330.0,312260.0,285070.0,NaN,NaN,NaN
Yemen,2020-01-01,29825967.0,15024985.0,14800983.0,219.749,174.558,42.9


In [27]:
group_rate.loc[:,'male_deaths'] = group_rate.adult_male_mortality_rate * group_rate.population_male/1000
group_rate.loc[:,'female_deaths'] = group_rate.adult_female_mortality_rate * group_rate.population_female/1000

In [28]:
group_rate

,date,population,population_male,population_female,adult_male_mortality_rate,adult_female_mortality_rate,infant_mortality_rate,male_deaths,female_deaths
country_name,,,,,,,,,
Afghanistan,2020-01-01,38928341.0,19976265.0,18952076.0,237.554,192.532,47.9,4.745442e+06,3.648881e+06
Albania,2020-01-01,2862427.0,1429594.0,1432833.0,93.315,49.486,7.8,1.334026e+05,7.090517e+04
Algeria,2020-01-01,43851043.0,22153807.0,21697235.0,102.851,80.672,20.1,2.278541e+06,1.750359e+06
American Samoa,2020-01-01,55196.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Andorra,2020-01-01,77265.0,58625.0,55581.0,NaN,NaN,2.7,NaN,NaN
...,...,...,...,...,...,...,...,...,...
Wallis and Futuna,2020-01-01,11245.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Western Sahara,2020-01-01,597330.0,312260.0,285070.0,NaN,NaN,NaN,NaN,NaN
Yemen,2020-01-01,29825967.0,15024985.0,14800983.0,219.749,174.558,42.9,3.301725e+06,2.583630e+06


In [29]:
fig = go.Figure(go.Bar(
            y=group_rate.population_male,
            x=group_rate.index,
            marker=dict(color= '#123456'),name='Población total'))
fig.add_trace(go.Bar(
            y=group_rate.male_deaths,
            x=group_rate.index,
            marker=dict(color= 'red'),name='muertes'))
fig.show()

In [30]:
fig = go.Figure(go.Bar(
            y=group_rate.population_female,
            x=group_rate.index,
            marker=dict(color= '#123456'),name='Población total'))
fig.add_trace(go.Bar(
            y=group_rate.female_deaths,
            x=group_rate.index,
            marker=dict(color= 'red'),name='muertes'))
fig.show()

In [31]:
fig = go.Figure(go.Bar(
            y=group_rate.population,
            x=group_rate.index,
            marker=dict(color= '#123456'),name='Población total'))
fig.show()

In [32]:
X = group.loc[:,['population_rural', 'population_urban', 
       'population_age_00_09', 'population_age_10_19', 'population_age_20_29',
       'population_age_30_39', 'population_age_40_49', 'population_age_50_59',
       'population_age_60_69', 'population_age_70_79', 'new_confirmed_age_0', 'new_confirmed_age_1',
       'new_confirmed_age_2', 'new_confirmed_age_3', 'new_confirmed_age_4',
       'new_confirmed_age_5', 'new_confirmed_age_6', 'new_confirmed_age_7',
       'new_confirmed_age_8', 'new_confirmed_age_9', 'new_deceased_age_0',
       'new_deceased_age_1', 'new_deceased_age_2', 'new_deceased_age_3',
       'new_deceased_age_4', 'new_deceased_age_5', 'new_deceased_age_6',
       'new_deceased_age_7', 'new_deceased_age_8', 'new_deceased_age_9',
       'new_hospitalized_patients_age_0', 'new_hospitalized_patients_age_1',
       'new_hospitalized_patients_age_2', 'new_hospitalized_patients_age_3',
       'new_hospitalized_patients_age_4', 'new_hospitalized_patients_age_5',
       'new_hospitalized_patients_age_6', 'new_hospitalized_patients_age_7',
       'new_hospitalized_patients_age_8', 'new_hospitalized_patients_age_9',
       'new_deceased_male', 'new_deceased_female', 'new_recovered_age_0',
       'new_recovered_age_1', 'new_recovered_age_2', 'new_recovered_age_3',
       'new_recovered_age_4', 'new_recovered_age_5', 'new_recovered_age_6',
       'new_recovered_age_7', 'new_recovered_age_8', 'new_recovered_age_9']]

In [33]:
X.loc[:,['population_rural','population_urban', 
       'population_age_00_09', 'population_age_10_19', 'population_age_20_29',
       'population_age_30_39', 'population_age_40_49', 'population_age_50_59',
       'population_age_60_69', 'population_age_70_79', 'new_confirmed_age_0',]] = X.loc[:,['population_rural','population_urban', 
       'population_age_00_09', 'population_age_10_19', 'population_age_20_29',
       'population_age_30_39', 'population_age_40_49', 'population_age_50_59',
       'population_age_60_69', 'population_age_70_79', 'new_confirmed_age_0',]]/707

In [34]:
for col in X.columns:
  X.loc[:,col]= X.loc[:,col] / group.population

In [35]:
X = X.dropna()

In [36]:
kmeans = KMeans(n_clusters=5).fit(X.dropna())
centroids = kmeans.cluster_centers_
print(centroids)

[[ 1.86417360e-04  1.24104420e-03  1.84767720e-04  1.81940604e-04
   2.00660127e-04  2.13483605e-04  1.91885952e-04  1.73721032e-04
   1.30779085e-04  7.93985587e-05  1.59723083e-09  2.07454510e-06
   3.60819882e-06  3.97489656e-06  3.37531979e-06  2.49937543e-06
   1.51666371e-06  8.59631374e-07  4.69665046e-07  1.02912001e-07
   9.00024837e-10  9.16396643e-10  4.61601658e-09  1.48836115e-08
   3.90443998e-08  7.68844306e-08  9.75013840e-08  1.12154063e-07
   9.86801638e-08  2.30567164e-08  1.98255218e-08  1.23756358e-08
   3.37577308e-08  6.04269137e-08  8.56093014e-08  1.08831671e-07
   1.16038447e-07  1.03176128e-07  8.89130663e-08  1.36552703e-08
   2.66280322e-07  2.01241873e-07  3.40278202e-07  6.47265762e-07
   1.16143484e-06  1.37414047e-06  1.15614323e-06  8.28612979e-07
   4.73103175e-07  2.74535941e-07  1.09554218e-07  2.52950432e-08]
 [ 9.65303523e-04  4.24391952e-04  3.42357412e-04  2.91029331e-04
   2.41943228e-04  1.89410913e-04  1.38195655e-04  1.04311432e-04
   6.7526

In [37]:
X.loc[:,'cluster'] = kmeans.predict(X)

In [38]:
X[X.cluster==0].index

Index(['American Samoa', 'Andorra', 'Argentina', 'Australia', 'Bahamas',
       'Bahrain', 'Belarus', 'Belgium', 'Bermuda', 'Brazil', 'Brunei',
       'Bulgaria', 'Canada', 'Cayman Islands', 'Chile', 'Colombia',
       'Costa Rica', 'Cuba', 'Curaçao', 'Cyprus', 'Czech Republic', 'Denmark',
       'Djibouti', 'Dominican Republic', 'Finland', 'France', 'Gabon',
       'Germany', 'Gibraltar', 'Greece', 'Greenland', 'Guam', 'Hong Kong',
       'Iceland', 'Iran', 'Israel', 'Japan', 'Jordan', 'Kuwait', 'Lebanon',
       'Libya', 'Luxembourg', 'Macau', 'Malaysia', 'Malta', 'Marshall Islands',
       'Mexico', 'Monaco', 'Nauru', 'Netherlands', 'New Zealand',
       'Northern Mariana Islands', 'Norway', 'Oman', 'Palau', 'Peru',
       'Puerto Rico', 'Qatar', 'Russia', 'San Marino', 'Saudi Arabia',
       'Singapore', 'Sint Maarten', 'South Korea', 'Spain', 'Sweden',
       'Switzerland', 'Turkey', 'Turks and Caicos Islands',
       'United Arab Emirates', 'United Kingdom',
       'United States

In [39]:
fig = go.Figure(go.Bar(
            y=X.loc[X[X.cluster==0].index, 'population_rural'],
            x=X[X.cluster==0].index,
            marker=dict(color= '#123456'),name='Población total'))
fig.show()

In [40]:
fig = go.Figure(go.Bar(
            y=X.loc[X[X.cluster==0].index, 'population_urban'],
            x=X[X.cluster==0].index,
            marker=dict(color= '#123456'),name='Población total'))
fig.show()

In [41]:
fig = go.Figure(go.Bar(
            y=X.loc[X[X.cluster==1].index, 'population_age_70_79'],
            x=X[X.cluster==1].index,
            marker=dict(color= '#123456'),name='Población total'))
fig.show()

In [42]:
fig = go.Figure(go.Bar(
            y=X.loc[X[X.cluster==0].index, 'population_age_00_09'],
            x=X[X.cluster==0].index,
            marker=dict(color= '#123456'),name='Población total'))
fig.show()

In [43]:
fig = go.Figure(go.Bar(
            y=X.loc[X[X.cluster==1].index, 'population_urban'],
            x=X[X.cluster==1].index,
            marker=dict(color= '#123456'),name='Población total'))
fig.show()

In [44]:
fig = go.Figure(go.Bar(
            y=X.loc[X[X.cluster==3].index, 'population_rural'],
            x=X[X.cluster==3].index,
            marker=dict(color= '#123456'),name='Población total'))
fig.show()

In [45]:
fig = go.Figure(go.Bar(
            y=X.loc[X[X.cluster==4].index, 'population_urban'],
            x=X[X.cluster==4].index,
            marker=dict(color= '#123456'),name='Población total'))
fig.show()

# Piramides clusterizacion


In [49]:
Cluster_0 = ['Albania', 'Algeria', 'Andorra', 'Antigua and Barbuda',
       'Argentina', 'Armenia', 'Australia', 'Austria', 'Azerbaijan',
       'Bahrain', 'Barbados', 'Belarus', 'Belgium', 'Belize',
       'Bosnia and Herzegovina', 'Brazil', 'Bulgaria', 'Canada', 'Chile',
       'China', 'Colombia', 'Costa Rica', 'Croatia', 'Cuba', 'Cyprus',
       'Denmark', 'Dominica', 'Dominican Republic', 'Ecuador',
       'El Salvador', 'Estonia', 'Fiji', 'Finland', 'France', 'Georgia',
       'Germany', 'Greece', 'Grenada', 'Guyana', 'Hungary', 'Iceland',
       'Ireland', 'Israel', 'Italy', 'Jamaica', 'Japan', 'Jordan',
       'Kazakhstan', 'Kuwait', 'Latvia', 'Lebanon', 'Liechtenstein',
       'Lithuania', 'Luxembourg', 'Malaysia', 'Maldives', 'Malta',
       'Mauritius', 'Mexico', 'Moldova', 'Monaco', 'Mongolia',
       'Montenegro', 'Morocco', 'Nauru', 'Netherlands', 'New Zealand',
       'Norway', 'Oman', 'Palau', 'Panama', 'Paraguay', 'Peru', 'Poland',
       'Portugal', 'Qatar', 'Romania', 'Russian Federation', 'Samoa',
       'San Marino', 'Saudi Arabia', 'Serbia', 'Seychelles', 'Singapore',
       'Slovenia', 'Spain', 'Sri Lanka', 'Suriname', 'Sweden',
       'Switzerland', 'Thailand', 'Tonga', 'Trinidad and Tobago',
       'Tunisia', 'Turkey', 'Ukraine', 'United Arab Emirates',
       'United Kingdom', 'Uruguay', 'Uzbekistan', 'Vietnam']

In [50]:
Cluster_1 = ['Afghanistan', 'Angola', 'Bangladesh', 'Benin', 'Bhutan',
       'Bolivia', 'Botswana', 'Burundi', 'Cambodia', 'Cameroon',
       'Central African Republic', 'Chad', 'Comoros', 'Djibouti',
       'Equatorial Guinea', 'Eritrea', 'Ethiopia', 'Gabon', 'Ghana',
       'Guatemala', 'Guinea', 'Guinea-Bissau', 'Haiti', 'Honduras',
       'India', 'Indonesia', 'Iraq', 'Kenya', 'Kiribati', 'Lesotho',
       'Liberia', 'Libya', 'Madagascar', 'Malawi', 'Mali',
       'Marshall Islands', 'Mauritania', 'Mozambique', 'Namibia', 'Nepal',
       'Nicaragua', 'Niger', 'Nigeria', 'Pakistan', 'Papua New Guinea',
       'Philippines', 'Rwanda', 'Sao Tome and Principe', 'Senegal',
       'Sierra Leone', 'Solomon Islands', 'Somalia', 'South Africa',
       'South Sudan', 'Sudan', 'Tajikistan', 'Tanzania', 'Togo',
       'Turkmenistan', 'Tuvalu', 'Uganda', 'Vanuatu', 'Zambia',
       'Zimbabwe']

In [51]:
new_confirmed = group.iloc[:,43:53][group.new_confirmed_age_4 !=0]
AgeClass = ['0-9','10-19', '20-29','30-39','40-49','50-59','60-69','70-79','80-89','90+']
new_confirmed.loc[:,'ochenta_mas'] = new_confirmed.new_confirmed_age_8	 + new_confirmed.new_confirmed_age_9
new_confirmed

,new_confirmed_age_0,new_confirmed_age_1,new_confirmed_age_2,new_confirmed_age_3,new_confirmed_age_4,new_confirmed_age_5,new_confirmed_age_6,new_confirmed_age_7,new_confirmed_age_8,new_confirmed_age_9,ochenta_mas
country_name,,,,,,,,,,,
Argentina,131102.0,417201.0,1082889.0,1213884.0,1027630.0,706784.0,418378.0,215182.0,96337.0,30053.0,126390.0
Brazil,891485.0,1122459.0,3112221.0,3970246.0,3382726.0,2433207.0,1392404.0,641819.0,252615.0,63043.0,315658.0
Colombia,174762.0,389758.0,1078198.0,1137278.0,860235.0,698339.0,420897.0,206197.0,94605.0,20795.0,115400.0
Czech Republic,175287.0,275102.0,257035.0,337260.0,447025.0,323357.0,210848.0,142517.0,62287.0,15958.0,78245.0
Estonia,15215.0,30197.0,26636.0,40762.0,37029.0,29611.0,23845.0,11220.0,4410.0,0.0,4410.0
Finland,23240.0,32056.0,39094.0,35148.0,27681.0,18765.0,9815.0,4928.0,3738.0,0.0,3738.0
France,442199.0,1079958.0,1339956.0,1249608.0,1131151.0,972615.0,649900.0,391816.0,263711.0,138722.0,402433.0
Germany,173323.0,734232.0,1813864.0,2284900.0,804326.0,371117.0,0.0,0.0,0.0,0.0,0.0
Hong Kong,545.0,761.0,1864.0,2310.0,2075.0,1936.0,1759.0,785.0,345.0,87.0,432.0


In [52]:
paises_cluster_confirm = []
for pais in group.new_confirmed_age_1[group.new_confirmed_age_1 !=0].index:
  if pais in Cluster_0:
    paises_cluster_confirm.append(pais)
paises_cluster_confirm

['Argentina',
 'Brazil',
 'Colombia',
 'Estonia',
 'Finland',
 'France',
 'Germany',
 'Malaysia',
 'Mexico',
 'New Zealand',
 'Peru',
 'Romania',
 'Spain',
 'Thailand']

In [53]:
def plot_bar_sub_cluster(paises, data_pob, data_new, AgeClass, title, color, labelx):
  pob = data_pob.loc[paises,:]
  new = data_new.loc[paises].iloc[:,[0,1,2,3,4,5,6,7,-1]]
  for pais in paises:
    pob.loc[pais,:] = pob.loc[pais,:].values/pob.sum(axis=1)[pais]

  for pais in paises:
    new.loc[pais,:] = new.loc[pais,:].values/new.sum(axis=1)[pais]

  fig = go.Figure(go.Bar(
              x=pob.mean(),
              y=AgeClass,
              orientation='h',
              marker=dict(color= '#123456'),name='Population rate'))

  fig.add_trace(go.Bar(
              x=new.mean(),
              y=AgeClass,
              orientation='h',
              marker=dict(color= color), name= labelx))
  
  fig.update_layout(title= title , #barmode='relative' f'Nuevos casos de Covid-19 por edad en el país {pais}'
                    xaxis_title='Population',
                    yaxis_title='Ages', 
                    autosize=False,
                    width=600,
                    height=600,
                    margin=dict(
                        l=50,
                        r=50,
                        b=100,
                        t=100,
                        pad=4
                    ))
  fig.show()

In [54]:
plot_bar_sub_cluster(paises_cluster_confirm, group_pop.iloc[:,14:], new_confirmed, AgesClassPop, 'New cases of Covid-19 by ages in cluster 0', 'orange', 'New cases date')

In [55]:
paises_cluster1_confirm = []
for pais in new_confirmed.index:
  if pais in Cluster_1:
    paises_cluster1_confirm.append(pais)
paises_cluster1_confirm


['India', 'Philippines']

In [56]:
plot_bar_sub_cluster(paises_cluster1_confirm, group_pop.iloc[:,14:], new_confirmed, AgesClassPop, 'New cases of Covid-19 by ages in cluster 1', 'orange', 'New cases rate')

In [57]:
new_deceased = group.iloc[:,53:63][group.new_deceased_age_4 !=0]
new_deceased.loc[:,'ochenta_mas'] = new_deceased.new_deceased_age_8	 + new_deceased.new_deceased_age_9
new_deceased

,new_deceased_age_0,new_deceased_age_1,new_deceased_age_2,new_deceased_age_3,new_deceased_age_4,new_deceased_age_5,new_deceased_age_6,new_deceased_age_7,new_deceased_age_8,new_deceased_age_9,ochenta_mas
country_name,,,,,,,,,,,
Argentina,136.0,167.0,836.0,2565.0,6912.0,14681.0,25699.0,30790.0,24812.0,10021.0,34833.0
Brazil,1161.0,1201.0,7309.0,25144.0,52128.0,90462.0,124924.0,124726.0,86508.0,26849.0,113357.0
Colombia,336.0,426.0,2297.0,5444.0,11252.0,22161.0,34424.0,36453.0,28576.0,8704.0,37280.0
Czech Republic,4.0,4.0,27.0,124.0,443.0,1311.0,4911.0,12091.0,11225.0,3628.0,14853.0
Germany,13.0,16.0,265.0,4975.0,31009.0,66835.0,0.0,0.0,0.0,0.0,0.0
Hong Kong,0.0,0.0,0.0,2.0,4.0,9.0,29.0,61.0,80.0,28.0,108.0
India,1.0,0.0,1.0,2.0,5.0,5.0,17.0,9.0,3.0,0.0,3.0
Malaysia,64.0,80.0,688.0,2269.0,4254.0,5935.0,7240.0,5824.0,3507.0,753.0,4260.0
Mexico,598.0,617.0,4333.0,13994.0,33506.0,60186.0,79651.0,65218.0,31589.0,5511.0,37100.0


In [58]:
paises_cluster_deceased0 = []
for pais in new_deceased.index:
  if pais in Cluster_0:
    paises_cluster_deceased0.append(pais)
paises_cluster_deceased0

['Argentina',
 'Brazil',
 'Colombia',
 'Germany',
 'Malaysia',
 'Mexico',
 'Peru',
 'Spain']

In [59]:
plot_bar_sub_cluster(paises_cluster_deceased0, group_pop.iloc[:,14:], new_deceased, AgesClassPop, 'Deceased of Covid-19 by age in cluster 0', 'red', 'Deaths rate')

In [60]:
paises_cluster_deceased1 = []
for pais in new_deceased.index:
  if pais in Cluster_1:
    paises_cluster_deceased1.append(pais)
paises_cluster_deceased1

['India', 'Philippines']

In [61]:
plot_bar_sub_cluster(paises_cluster_deceased1, group_pop.iloc[:,14:], new_deceased, AgesClassPop, 'Deceased of Covid-19 by age in cluster 1', 'red', 'Deaths rate')

In [62]:
new_hospitalized = group.iloc[:,63:73][group.new_hospitalized_patients_age_4 !=0]
new_hospitalized.loc[:,'ochenta_mas'] = new_hospitalized.new_hospitalized_patients_age_8	 + new_hospitalized.new_hospitalized_patients_age_9
new_hospitalized

,new_hospitalized_patients_age_0,new_hospitalized_patients_age_1,new_hospitalized_patients_age_2,new_hospitalized_patients_age_3,new_hospitalized_patients_age_4,new_hospitalized_patients_age_5,new_hospitalized_patients_age_6,new_hospitalized_patients_age_7,new_hospitalized_patients_age_8,new_hospitalized_patients_age_9,ochenta_mas
country_name,,,,,,,,,,,
Argentina,33958.0,15218.0,29295.0,40600.0,47151.0,55771.0,65471.0,66650.0,51590.0,20266.0,71856.0
Brazil,19889.0,14433.0,70424.0,191585.0,285052.0,353061.0,338209.0,267812.0,156345.0,42064.0,198409.0
Hong Kong,539.0,756.0,1840.0,2282.0,2055.0,1917.0,1719.0,720.0,264.0,59.0,323.0
India,78.0,194.0,463.0,508.0,362.0,283.0,208.0,59.0,13.0,4.0,17.0
Mexico,6354.0,5881.0,26357.0,56369.0,94177.0,129836.0,136302.0,98055.0,44398.0,7474.0,51872.0
Philippines,2339.0,3313.0,12394.0,13343.0,11442.0,13857.0,13347.0,8284.0,3197.0,480.0,3677.0
Spain,3421.0,4499.0,15203.0,27557.0,48183.0,67743.0,77333.0,82063.0,112454.0,0.0,112454.0


In [63]:
paises_cluster_hospitalized0 = []
for pais in new_hospitalized.index:
  if pais in Cluster_0:
    paises_cluster_hospitalized0.append(pais)
paises_cluster_hospitalized0

['Argentina', 'Brazil', 'Mexico', 'Spain']

In [64]:
plot_bar_sub_cluster(paises_cluster_hospitalized0, group_pop.iloc[:,14:], new_hospitalized, AgesClassPop, 'Hospitalized Covid-19 by ages cluster 0', 'dodgerblue', 'Hospitalized rate')

In [65]:
paises_cluster_hospitalized1 = []
for pais in new_hospitalized.index:
  if pais in Cluster_1:
    paises_cluster_hospitalized1.append(pais)
paises_cluster_hospitalized1

['India', 'Philippines']

In [66]:
plot_bar_sub_cluster(paises_cluster_hospitalized1, group_pop.iloc[:,14:], new_hospitalized, AgesClassPop, 'Hospitalized Covid-19 by ages cluster 1', 'dodgerblue', 'Hospitalized rate')

In [67]:
new_recovered = group.iloc[:,75:85][group.new_recovered_age_3 !=0]
new_recovered.loc[:,'ochenta_mas'] = new_recovered.new_recovered_age_8	 + new_recovered.new_recovered_age_9
new_recovered

,new_recovered_age_0,new_recovered_age_1,new_recovered_age_2,new_recovered_age_3,new_recovered_age_4,new_recovered_age_5,new_recovered_age_6,new_recovered_age_7,new_recovered_age_8,new_recovered_age_9,ochenta_mas
country_name,,,,,,,,,,,
Colombia,173658.0,388494.0,1073676.0,1129336.0,847101.0,674895.0,385886.0,169737.0,66330.0,12199.0,78529.0
Czech Republic,137074.0,213810.0,229766.0,292786.0,391235.0,291930.0,186398.0,119506.0,47983.0,11731.0,59714.0
Germany,146057.0,548738.0,1569850.0,1924122.0,657518.0,270786.0,0.0,0.0,0.0,0.0,0.0
Philippines,17570.0,31119.0,105384.0,94532.0,63751.0,49696.0,29583.0,12630.0,3981.0,509.0,4490.0


In [68]:
paises_cluster_recovered0 = []
for pais in new_recovered.index:
  if pais in Cluster_0:
    paises_cluster_recovered0.append(pais)
paises_cluster_recovered0

['Colombia', 'Germany']

In [69]:
plot_bar_sub_cluster(paises_cluster_recovered0, group_pop.iloc[:,14:], new_recovered, AgesClassPop, 'Recovered of Covid-19 by age in cluster 0', '#5DB009', 'Recovered rate')

In [70]:
paises_cluster_recovered1 = []
for pais in new_recovered.index:
  if pais in Cluster_1:
    paises_cluster_recovered1.append(pais)
paises_cluster_recovered1

['Philippines']

In [71]:
plot_bar_sub_cluster(paises_cluster_recovered1, group_pop.iloc[:,14:], new_recovered, AgesClassPop, 'Recovered of Covid-19 by age in cluster 1', '#5DB009', 'Recovered')

# Otros

In [72]:
new_confirmed.index

Index(['Argentina', 'Brazil', 'Colombia', 'Czech Republic', 'Estonia',
       'Finland', 'France', 'Germany', 'Hong Kong', 'India', 'Malaysia',
       'Mexico', 'New Zealand', 'Peru', 'Philippines', 'Romania', 'Spain',
       'Taiwan', 'Thailand'],
      dtype='object', name='country_name')

In [73]:
new_deceased.index


Index(['Argentina', 'Brazil', 'Colombia', 'Czech Republic', 'Germany',
       'Hong Kong', 'India', 'Malaysia', 'Mexico', 'Peru', 'Philippines',
       'Spain'],
      dtype='object', name='country_name')

In [74]:
new_hospitalized.index

Index(['Argentina', 'Brazil', 'Hong Kong', 'India', 'Mexico', 'Philippines',
       'Spain'],
      dtype='object', name='country_name')

In [75]:
new_recovered.index

Index(['Colombia', 'Czech Republic', 'Germany', 'Philippines'], dtype='object', name='country_name')